# CS 3110/5110: Data Privacy
## In-Class Exercise, week of 10/30/2023

In [1]:
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

# adult = pd.read_csv('https://github.com/jnear/cs211-data-privacy/raw/master/homework/adult_with_pii.csv')

In [2]:
# Load data files
import numpy as np
import urllib.request
import io

url_x = 'https://github.com/jnear/cs211-data-privacy/raw/master/slides/adult_processed_x.npy'
url_y = 'https://github.com/jnear/cs211-data-privacy/raw/master/slides/adult_processed_y.npy'

with urllib.request.urlopen(url_x) as url:
    f = io.BytesIO(url.read())
X = np.load(f)

with urllib.request.urlopen(url_y) as url:
    f = io.BytesIO(url.read())
y = np.load(f)

In [3]:
# Split data into training and test sets
training_size = int(X.shape[0] * 0.8)

X_train = X[:training_size]
X_test = X[training_size:]

y_train = y[:training_size]
y_test = y[training_size:]

print('Train and test set sizes:', len(y_train), len(y_test))

Train and test set sizes: 36176 9044


## Question 1

Using scikit-learn, train a logistic regression model on the training data loaded above.

In [4]:
from sklearn.linear_model import LogisticRegression

In [5]:
def train_model():
    model = LogisticRegression().fit(X_train, y_train)
    return model

model = train_model()
print('Model coefficients:', model.coef_[0])
print('Model accuracy:', np.sum(model.predict(X_test) == y_test)/X_test.shape[0])

Model coefficients: [ 4.78170133e-01 -1.79931321e-01 -3.40133603e-02  6.84523207e-02
 -5.91006819e-01 -3.66105390e-01 -1.06084670e+00 -6.69733212e-01
 -6.36881044e-01 -4.60609540e-01 -5.10585899e-01 -3.97823096e-01
 -7.57646283e-01 -7.81249016e-01  6.35796558e-02  1.18071728e-01
  5.10755783e-01  1.00306576e+00 -1.12271824e-01  7.38195557e-01
 -1.18449574e+00  1.28199885e+00  1.10347188e-01 -8.97886249e-01
  1.50204541e+00  1.25197643e+00 -5.83464324e-01 -1.31378111e+00
 -8.89740089e-01 -7.54431205e-01 -5.67395184e-02  5.14558992e-02
 -8.00528755e-04  7.21861649e-01 -8.96547192e-01 -6.94310123e-01
 -3.23898536e-01 -9.41107325e-01 -1.09689047e+00  4.72878928e-01
  4.76955030e-01  2.21566753e-01  5.09572280e-01 -1.29277978e-01
 -3.74288894e-01  1.81115326e-03 -7.74189090e-01 -1.05094022e+00
 -1.90642329e-01  7.02968244e-01 -5.02458284e-01 -4.50935879e-02
 -4.88093508e-01 -4.18043863e-01 -2.31591890e-01 -1.17524508e+00
 -5.10036049e-01  8.94588254e-01  6.12571489e-01 -2.84250889e-01
 -1.2

C:\Users\npsul\anaconda3\envs\CS3110-Data-Privacy\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Question 2

Implement the *average gradient* of the loss below.

In [6]:
# The loss function measures how good our model is. The training goal is to minimize the loss.
# This is the logistic loss function.
def loss(theta, xi, yi):
    exponent = - yi * (xi.dot(theta))
    return np.log(1 + np.exp(exponent))

# This is the gradient of the logistic loss
# The gradient is a vector that indicates the rate of change of the loss in each direction
def gradient(theta, xi, yi):
    exponent = yi * (xi.dot(theta))
    return - (yi*xi) / (1+np.exp(exponent))

In [9]:
def avg_grad(theta, X, y):
    example_grads = [gradient(theta, xi, yi) for xi, yi in zip(X, y)]
    avg_grad = np.mean(example_grads, axis=0)
    return avg_grad

## Question 3

Use the average gradient from above to implement a gradient descent algorithm.

In [10]:
def gradient_descent(iterations):
    theta = np.zeros(104)
    
    for i in range(iterations):
        # calculate the gradient
        grad = avg_grad(theta, X_train, y_train)
        # update the model by moving in the opposite direction of the gradient
        theta = theta + -grad
    return theta

theta = gradient_descent(10)
theta

array([ 1.63933476e-02, -2.62737908e-02, -3.76703876e-01,  5.75414219e-02,
       -6.45755008e-02, -2.73486282e-02, -1.30672661e-03, -5.91543132e-02,
       -7.64880798e-02, -2.62061378e-02, -1.57561761e-02, -2.86662073e-02,
       -4.72301785e-02, -3.76260473e-02, -9.90166218e-03, -1.97245917e-02,
        1.55177596e-01,  4.49929106e-02, -3.29796604e-01,  1.19518106e-01,
       -4.89454322e-03,  6.88790663e-02, -1.55396891e-01, -1.80599005e-01,
        1.54260151e-03,  3.89119966e-01, -1.97894731e-02, -5.15748994e-01,
       -5.58626968e-02, -4.09361505e-02, -1.28065226e-01, -1.62203385e-04,
       -1.08119029e-01,  1.88498244e-01, -6.47481586e-02, -8.66006277e-02,
       -9.99400278e-02, -2.05565204e-01, -1.11388992e-02,  1.61706332e-01,
       -3.59862313e-03, -1.50012146e-02,  1.68869178e-03, -5.12278071e-02,
        3.20211424e-01, -2.99730645e-01, -6.21807885e-02, -2.79987375e-01,
       -1.81265739e-01,  8.06793703e-02, -1.62147320e-02, -2.28589562e-02,
       -1.37829430e-01, -

In [11]:
# Prediction: take a model (theta) and a single example (xi) and return its predicted label
def predict(xi, theta, bias=0):
    label = np.sign(xi @ theta + bias)
    return label

def accuracy(theta):
    return np.sum(predict(X_test, theta) == y_test)/X_test.shape[0]

accuracy(theta)

0.7787483414418399

## Question 4

Implement a *noisy gradient descent* algorithm.

1. Calculate gradients for each example
2. Clip the gradients to have bounded $L2$ norm
3. Sum the clipped gradients
4. Use the Gaussian mechanism to add noise to the sum of gradients

In [12]:
def L2_clip(v, b):
    norm = np.linalg.norm(v, ord=2)
    
    if norm > b:
        return b * (v / norm)
    else:
        return v

def noisy_gradient_descent(iterations, epsilon, delta):
    theta = np.zeros(104)
    noisy_count = laplace_mech(len(X_train), 1, epsilon)
    
    for i in range(iterations):
        # calculate the gradient
        example_grads = [gradient(theta, xi, yi) for xi, yi in zip(X_train, y_train)]
        
        # enforces L2 bound on each gradient
        clipped_grads = [L2_clip(g, 3) for g in example_grads]
        grad_sum = np.sum(clipped_grads, axis=0)  # L2 sensitivity of 3
        
        # Gaussian used because it lets us take advantage of L2 sensitivity
        noisy_grad = gaussian_mech_vec(grad_sum, 3, epsilon, delta)
        
        # update the model by moving in the opposite direction of the gradient
        theta = theta + - np.array(noisy_grad) / noisy_count
        
    return theta

theta = noisy_gradient_descent(10, 1.0, 1e-5)
print('Final accuracy:', accuracy(theta))

Final accuracy: 0.7786377708978328


In [13]:
# TEST CASE

assert accuracy(noisy_gradient_descent(5, 0.001, 1e-5)) < 0.76
assert accuracy(noisy_gradient_descent(5, 1.0, 1e-5)) > 0.70

## Question 5

What is the *total privacy cost* of the noisy gradient descent algorithm above, and why? Argue informally that the algorithm satisfies this privacy cost. Use sequential composition.

1. Sensitivity
    - Laplace mech: it's a counting query so the sensitivity is 1
    - Gaussian mech: it's a sum of clipped vectors. Each vector in the sum is clipped to have L2 norm of at most 3. Therefore, the L2 sensitivity of the sum is 3
2. Composition
    - We call the Laplace mech one and the Gaussian mech 10 times so the total privacy cost is 10epsilon + epsilon, 10delta
3. Post-Processing
    - We start with an initial theta that is independent of the data (all 0s) and each update to theta is based on differentially private values. So the final model satisfies differential privacy by post-processing.

## Question 6

Repeat the above, but using advanced composition.

By advanced composition the total epsilon is 2epsilon*sqrt(2k*log(1/delta'))
- Here k = 10
- I set delta' = 1e-5
- So the total epsilon for the Gaussian mechanism is 2epsilon*sqrt(2*10*log(1/1e-5))
- The total epsilon for the whole training procedure is 2epsilon*sqrt(2*10*log(1/1e-5)) + epsilon
- The total delta is k*delta + delta' = 10delta + 1e5

## Question 7

Implement a version of noisy gradient descent that satisfies a *total* of $(\epsilon, \delta)$-differential privacy. Use sequential composition.

In [14]:
def noisy_gradient_descent(iterations, epsilon, delta):
    theta = np.zeros(104)
    epsilon_i = epsilon / (iterations + 1)
    delta_i = delta / iterations
    noisy_count = laplace_mech(len(X_train), 1, epsilon)
    
    for i in range(iterations):
        # calculate the gradient
        example_grads = [gradient(theta, xi, yi) for xi, yi in zip(X_train, y_train)]
        
        # enforces L2 bound on each gradient
        clipped_grads = [L2_clip(g, 3) for g in example_grads]
        grad_sum = np.sum(clipped_grads, axis=0)  # L2 sensitivity of 3
        
        # Gaussian used because it lets us take advantage of L2 sensitivity
        noisy_grad = gaussian_mech_vec(grad_sum, 3, epsilon_i, delta_i)
        
        # update the model by moving in the opposite direction of the gradient
        theta = theta + - np.array(noisy_grad) / noisy_count
        
    return theta

theta = noisy_gradient_descent(10, 1.0, 1e-5)
print('Final accuracy:', accuracy(theta))

Final accuracy: 0.777421494913755


In [15]:
# TEST CASE

assert accuracy(noisy_gradient_descent(5, 0.001, 1e-5)) < 0.76
assert accuracy(noisy_gradient_descent(5, 1.0, 1e-5)) > 0.70